In [9]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages')
sys.path

['/home/.schramm/Documents/3d_retrieval',
 '/opt/anaconda3/lib/python37.zip',
 '/opt/anaconda3/lib/python3.7',
 '/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda3/lib/python3.7/site-packages',
 '/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/home/.schramm/.ipython',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/local/lib/python2.7/dist-packages']

In [11]:
import create_dict
import torch
import os
import json
import utils

# Clean Data

In [ ]:
folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
input_folder = '/data_cleaned'
suffix = '_test' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

# Initialize Network

In [14]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_test' # comment in if not coming from generating the dataset
path_to_params = "models/test.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

In [15]:
device

device(type='cuda', index=0)

# Train Network

In [16]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [17]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

63 train triplets generated -> return dict
Loaded 5032 word vectors.
Number of training triplets: 63
[1,     2] loss: 2.713
[1,     3] loss: 0.194
[1,     4] loss: 0.406
[1,     5] loss: 0.703
[1,     6] loss: 0.250
[1,     7] loss: 0.817
[1,     8] loss: 0.240
[1,     9] loss: 1.344
[1,    10] loss: 3.194
[1,    11] loss: 0.250
[1,    12] loss: 0.255
[1,    13] loss: 0.251
[1,    14] loss: 1.925
[1,    15] loss: 0.593
[1,    16] loss: 0.826
[1,    17] loss: 1.841
[1,    18] loss: 0.250
[1,    19] loss: 0.339
[1,    20] loss: 0.248
[1,    21] loss: 0.254
[1,    22] loss: 1.997
[1,    23] loss: 0.521
[1,    24] loss: 0.250
[1,    25] loss: 0.664
[1,    26] loss: 1.453
[1,    27] loss: 1.089
[1,    28] loss: 1.330
[1,    29] loss: 1.762
[1,    30] loss: 1.090
[1,    31] loss: 0.240
[1,    32] loss: 0.252
[1,    33] loss: 0.246
[1,    34] loss: 0.800
[1,    35] loss: 1.351
[1,    36] loss: 0.330
[1,    37] loss: 0.301
[1,    38] loss: 0.251
[1,    39] loss: 0.250
[1,    40] loss: 0.248
[1

# Validation

In [ ]:
margin = 0.5
writer_suffix = 'Val'
SiameseNet.val(net, margin, data_dir_val, writer_suffix, working_dir, class_dir)

# Retrieval

In [18]:
y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir)

8 val triplets generated
Loaded 5032 word vectors.
Number of validation triplets: 4


TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
num_KNN = 4
max_show = 3
utils.retrieve_images(y_pred, ids, data_dir_val, class_dir, num_KNN, max_show)